In [44]:
# read fasta sequence and convert to string

from Bio import SeqIO
import arnie
from arnie.bpps import bpps
# from arnie.pfunc import pfunc COMMENT you never use this so you do not need to import
from arnie.mea.mea import MEA
from arnie.mea.mea_utils import *

# COMMENT, just convention: in general we put all the inputs at the top of the file

def get_seq(seq_filename):
    record = SeqIO.read(seq_filename, "fasta")
    full_seq = str(record.seq)
    return full_seq

def get_seq_RCK(seq_filename):
    record = SeqIO.read(seq_filename, "fasta")
    return str(record.seq)

# divide string into list of sliding windows of appropriate length

def get_sliding_windows(full_seq, step, window):
    windows = []
    for i in range(0, len(full_seq)+1, step):
        new_windows = full_seq[i:i+window]
        if len(new_windows) == window: # COMMENT so this works, the only thing I did different, is instead
            # of checking that each is the correct length, we know the only ones that will be too short
            # are the end ones so instead of going all the way to end in range, I stopped once we
            # were less than a window away from the end.
            windows.append(new_windows)
    return windows

def get_sliding_windows_RCK(full_seq, step, window):
    coords = list(range(0,len(full_seq)-window+1,step))
    windows = [full_seq[i:i+window] for i in coords]
    return windows, coords

# for loop to run bpps on each item in the list
def get_bpps_list(windows):
    bpps_list = []
    # COMMENT it is good practice (one I often forget) to not always use "i" in for loops
    # but instead something descriptive and i reserved for when we are looping through indexes
    # so for example as we loop through windows we are looking at sequence for you can do
    # for seq in windows instead so it is clearer what "i"/"seq" is
    for i in windows:
        new_bpps = bpps(i, package='contrafold', linear=True, threshknot=True)
        bpps_list.append(new_bpps)
    return bpps_list
#COMMENT just for descriptive purposes bpp is really a matrix (which is represented by a list of lists here)

def get_bpp_matrix_RCK(window):
    bpp = bpps(seq, package='contrafold', linear=True)#, threshknot=True)
    return bpp

# for loop to run MEA on each item to convert bpps to bp_list

def get_bp_list(bpps_list):
    bp_list = []
    # same comment as above eg for bpp in bpps_list:
    for i in bpps_list:
        new_bp = MEA(i, run_probknot_heuristic=True, theta=0.3)
        new_bp_list = new_bp.MEA_bp_list
        bp_list.append(new_bp_list)
    return bp_list

def get_bp_list_RCK(bpp):
    adj_matrix = MEA(bpp, run_probknot_heuristic=True)#, theta=0.3)
    bp_list = adj_matrix.MEA_bp_list
    return bp_list

# function to go over each bp_list and check whether or not it is a pseudoknot

def is_PK(bp_list):
    '''checks if a given bp_list represents a PK
    Args:
        bp_list: of list of tuples where the tuples are the indeces of the bp in increasing order (bp[0]<bp[1])
    
    returns:
        True if it is a psuedoknot
    '''
    bp_list = check_bp_list(bp_list)
    for i in bp_list:
        if bp_list == []:
            return False
        else:
            current_bp = bp_list[0]
            for bp in bp_list[1:]:
                if ((current_bp[0] < bp[0] and bp[0] < current_bp[1] and current_bp[1] < bp[1])
                    or (current_bp[0] > bp[0] and current_bp[0] < bp[1] and bp[1] < current_bp[1])):
                    return True
            return is_PK(bp_list[1:])
def check_bp_list(bp_list):
    for bp in bp_list:
        bp.sort()
    bp_list.sort(key = lambda x: x[0])
    nts = [nt for bp in bp_list for nt in bp]
    if len(nts)>len(set(nts)):
        print("WARNING some nucletotides found in more than 1 bp")
        for i,bpA in enumerate(bp_list):
            for bpB in bp_list[i+1:]:
                if bpA[0] == bpB[0] and bpA[1] == bpB[1]:
                    print("removing repeat bp",bpA)
                    bp_list = bp_list[:i]+bp_list[i+1:]
                elif bpA[0] in bpB:
                    print("ERROR base",bpA[0],"is in 2 basepairs",bpA,bpB,"THIS MUST BIC FIXED")
                    return None
                elif bpA[1] in bpB:
                    print("ERROR base",bpA[1],"is in 2 basepairs",bpA,bpB,"THIS MUST BIC FIXED")
                    return None
    return bp_list

def get_pseudoknots(seq_filename, step, window):
    full_seq = get_seq(seq_filename)
    windows = get_sliding_windows(full_seq, step=step, window=window)
    bpps_list = get_bpps_list(windows)
    bp_list = get_bp_list(bpps_list)
    PK_list = is_PK(bp_list)
    return PK_list

def get_pseudoknots_RCK(seq_filename, step, window):
    seq = get_seq_RCK(seq_filename)
    windows,coords = get_sliding_windows_RCK(seq, step=step, window=window)
    PK_list = []
    for seq,coord in zip(windows,coords):
        print(seq)
        bpp = get_bpp_matrix_RCK(seq)
        print(bpp)
        bp_list = get_bp_list_RCK(bpp)
        print(bp_list)
        if is_PK(bp_list):
            PK_list.append([seq,bp_list,coord,coord+window])
    return PK_list

In [45]:
get_pseudoknots_RCK("NC_045512.2.fasta",10000,200)

ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGT
[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 1.7563e-04 6.

TypeError: 'NoneType' object is not iterable

In [20]:
def get_dotbracket_list(bpps_list):
    dotbracket_list = []
    for i in bpps_list:
        new_dotbracket = MEA(i, run_probknot_heuristic=True, theta=0.3)
        new_structure = new_dotbracket.structure
        dotbracket_list.append(new_structure)
    return dotbracket_list

In [30]:
from Bio import SeqIO
import arnie
from arnie.bpps import bpps
from arnie.pfunc import pfunc
from arnie.mea.mea import MEA
from arnie.mea.mea_utils import *

def get_pseudoknots(seq_filename, step, window):
        
        full_seq = get_seq(seq_filename)
        
        windows = get_sliding_windows(full_seq, step=step, window=window)
        
        bpps_list = get_bpps_list(windows)
        
        bp_list = get_bp_list(bpps_list)
        
        PK_list = is_PK(bp_list)
        
        return PK_list

In [31]:
get_pseudoknots("SARS_last_1000.fasta", step = 100, window = 300)

True